In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates. All rights reserved.


This tutorial shows how to fit a volume given a set of views of a scene using differentiable volumetric rendering.

More specificially, this tutorial will explain how to:
1. 如何创建一个可微分的立体渲染器.
2. 如何创建一个立体模型（包括如何使用 Volumes 类）
3. 使用可微分的立体渲染器，根据图像拟合立体结构
4. 将预测的立体结构可视化。

## 0. 安装和导入模块
确保已安装 torch 和 torchvision。如果没有安装 pytorch3d，请使用以下代码安装。:

In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.7") and sys.platform.startswith("linux"):
        # 通过发布的 wheel 软件包安装 PyTorch3D
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{torch.__version__[0:5:2]}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # 通过源码安装 PyTorch3D
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
import os
import sys
import time
import json
import glob
import torch
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython import display

# 用于渲染的数据结构和函数
from pytorch3d.structures import Volumes
from pytorch3d.renderer import (
    FoVPerspectiveCameras, 
    VolumeRenderer,
    NDCGridRaysampler,
    EmissionAbsorptionRaymarcher
)
from pytorch3d.transforms import so3_exponential_map

# 为 demo utils 函数添加路径 
sys.path.append(os.path.abspath(''))
from utils.plot_image_grid import image_grid
from utils.generate_cow_renders import generate_cow_renders

# 准备可用设备
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

## 1. 生成场景及 mask 的图像

The following cell generates our training data.
It renders the cow mesh from the `fit_textured_mesh.ipynb` tutorial from several viewpoints and returns:
1. A batch of image and silhouette tensors that are produced by the cow mesh renderer.
2. A set of cameras corresponding to each render.

Note: For the purpose of this tutorial, which aims at explaining the details of volumetric rendering, we do not explain how the mesh rendering, implemented in the `generate_cow_renders` function, works. Please refer to `fit_textured_mesh.ipynb` for a detailed explanation of mesh rendering.

In [ ]:
target_cameras, target_images, target_silhouettes = generate_cow_renders(num_views=40)
print(f'Generated {len(target_images)} images/silhouettes/cameras.')

## 2. 初始化体积渲染器

The following initializes a volumetric renderer that emits a ray from each pixel of a target image and samples a set of uniformly-spaced points along the ray. At each ray-point, the corresponding density and color value is obtained by querying the corresponding location in the volumetric model of the scene (the model is described & instantiated in a later cell).

The renderer is composed of a *raymarcher* and a *raysampler*.
- The *raysampler* is responsible for emiting rays from image pixels and sampling the points along them. Here, we use the `NDCGridRaysampler` which follows the standard PyTorch3D coordinate grid convention (+X from right to left; +Y from bottom to top; +Z away from the user).
- The *raymarcher* takes the densities and colors sampled along each ray and renders each ray into a color and an opacity value of the ray's source pixel. Here we use the `EmissionAbsorptionRaymarcher` which implements the standard Emission-Absorption raymarching algorithm.

In [ ]:
# render_size 表示渲染图像各个边的像素大小，将其设置为与目标图像尺寸一致 
# 也就是说将其渲染成与基准图像一样的尺寸
# 渲染场景以（0,0,0）为中心，被限定在一个边长约等于 3.0 (国际单位)的边框内
render_size = target_images.shape[1]

volume_extent_world = 3.0

# 1) 实例化 raysampler
# 此处 NDCGridRaysampler 会生成一矩形图像网格的射线
# 其坐标遵循 pytorch3d 坐标规定
# 大致相当于每个体素都有一个射线点
# 由于此处设定的体积是 128^3，因此取样 n_pts_per_ray=150
# 进一步设置 min_depth=0.1，因为相机平面内的所有表面都超过了 0.1 单位
raysampler = NDCGridRaysampler(
    image_width=render_size,
    image_height=render_size,
    n_pts_per_ray=150,
    min_depth=0.1,
    max_depth=volume_extent_world,
)


# 2) 实例化 raymarcher.
# 此处用的是标准 EmissionAbsorptionRaymarcher
# 它会沿着每条射线前进
# 将每条射线都渲染成一个单一的 3D 颜色向量和一个不透明度标量
raymarcher = EmissionAbsorptionRaymarcher()

# 最后，用 raysampler 和 raymarcher 实例化体积渲染器
renderer = VolumeRenderer(
    raysampler=raysampler, raymarcher=raymarcher,
)

## 3. 初始化体积模型

接下来实例化场景的体积模型。这会使得 3D 空间量化为体积像素，其中每个体素都用体素 RGB 颜色的 3D 向量，和描述体素不透明度的密度标量（范围在[0-1]之间，数字越大不透明越高）来表示.

为了保证密度和颜色的取值范围在 [0-1] 之间，我们会在对数空间中表示体积颜色和密度。模型运行 forward 函数时，log-space 值会通过 sigmoid 函数传递，从而使得 log-space 值处于正确的取值范围。

此外， VolumeModel 还包含渲染器对象。这个对象在整个优化过程中保持不变。

本部分代码还定义了 huber 损失函数，它可以计算出渲染色和 mask 之间的差异。

In [ ]:
class VolumeModel(torch.nn.Module):
    def __init__(self, renderer, volume_size=[64] * 3, voxel_size=0.1):
        super().__init__()
        # 对 torch.sigmoid(self.log_colors)进行评估后
        # 得到的密度值接近于 0
        self.log_densities = torch.nn.Parameter(-4.0 * torch.ones(1, *volume_size))
        # 对 torch.sigmoid(self.log_colors)进行评估后 
        # 图像变为中性灰色
        self.log_colors = torch.nn.Parameter(torch.zeros(3, *volume_size))
        self._voxel_size = voxel_size
        # 同时存储渲染器模块
        self._renderer = renderer
        
    def forward(self, cameras):
        batch_size = cameras.R.shape[0]

        # 将 log-space 值转换为密度/颜色
        densities = torch.sigmoid(self.log_densities)
        colors = torch.sigmoid(self.log_colors)
        
        # 实例化体积对象，地扩展了 batch_size-times
        # 确保密度和颜色准确无误
        # 扩展了 batch_size-times
        volumes = Volumes(
            densities = densities[None].expand(
                batch_size, *self.log_densities.shape),
            features = colors[None].expand(
                batch_size, *self.log_colors.shape),
            voxel_size=self._voxel_size,
        )
        
        # 给定相机和体积
        # 运行渲染器并只返回第一个输出值
        # (第二个输出表示采样射线
        # 此处可以忽略).
        return self._renderer(cameras=cameras, volumes=volumes)[0]
    
# 辅助函数，用于评估
# 渲染的轮廓和颜色之间的 smooth L1（huber）损失
def huber(x, y, scaling=0.1):
    diff_sq = (x - y) ** 2
    loss = ((1 + diff_sq / (scaling**2)).clamp(1e-4).sqrt() - 1) * float(scaling)
    return loss

## 4. 体积拟合

这一步，我们用可微分渲染来进行体积拟合

为了拟合体积，我们从 target_camera 的视角进行渲染
并将渲染结果与观察到的 target_images 和 target_silhouettes 进行对比

这种对比是通过评估的 target_images/rendered_images 和
 target_silhouettes/rendered_silhouettes 之间的平均 huber（smooth-l1）误差来完成的。

In [ ]:
# 首先将所有相关变量移至正确的设备上
target_cameras = target_cameras.to(device)
target_images = target_images.to(device)
target_silhouettes = target_silhouettes.to(device)

# 实例化体积模型
# 此处使用的是一个 
# 边长= 128 的立方体的体积
# 体积中每个体素的大小设置为 volume_extent_world/volume_size s.t.
# 该体积表示以(0，0，0)为中心的 3D 边框所包围的空间
# 每个边的大小为 3
volume_size = 128
volume_model = VolumeModel(
    renderer,
    volume_size=[volume_size] * 3, 
    voxel_size = volume_extent_world / volume_size,
).to(device)

# 实例化 Adam 优化器。将学习率设置为 0.1
lr = 0.1
optimizer = torch.optim.Adam(volume_model.parameters(), lr=lr)

# 将 Adam 迭代 300，并在每个 minibatch 中随机抽取 10 张图像
batch_size = 10
n_iter = 300
for iteration in range(n_iter):

    # 运行完 75% 的迭代次数时
    # 将优化器的学习率降低 10 倍
    if iteration == round(n_iter * 0.75):
        print('Decreasing LR 10-fold ...')
        optimizer = torch.optim.Adam(
            volume_model.parameters(), lr=lr * 0.1
        )
    
    # 将优化器梯度设置为零
    optimizer.zero_grad()
    
    # 随机批次指数取样
    batch_idx = torch.randperm(len(target_cameras))[:batch_size]
    
    # 相机 minibatch 取样
    batch_cameras = FoVPerspectiveCameras(
        R = target_cameras.R[batch_idx], 
        T = target_cameras.T[batch_idx], 
        znear = target_cameras.znear[batch_idx],
        zfar = target_cameras.zfar[batch_idx],
        aspect_ratio = target_cameras.aspect_ratio[batch_idx],
        fov = target_cameras.fov[batch_idx],
        device = device,
    )
    
    # 评估体积模型
    rendered_images, rendered_silhouettes = volume_model(
        batch_cameras
    ).split([3, 1], dim=-1)
    
    # 计算轮廓误差
    # 并将其作为预测 mask
    # 和目标轮廓之间的平均 huber 损失
    sil_err = huber(
        rendered_silhouettes[..., 0], target_silhouettes[batch_idx],
    ).abs().mean()

    # 计算颜色误差
    # 并将其作为渲染颜色
    # 和基准图像之间的平均 huber 损失
    color_err = huber(
        rendered_images, target_images[batch_idx],
    ).abs().mean()
    
    # 优化损失是颜色和
    # 轮廓误差的简单相加
    loss = color_err + sil_err 
    
    # 打印当前的损失值
    if iteration % 10 == 0:
        print(
            f'Iteration {iteration:05d}:'
            + f' color_err = {float(color_err):1.2e}'
            + f' mask_err = {float(sil_err):1.2e}'
        )
    
    # 继续优化
    loss.backward()
    optimizer.step()
    
    # 每跑完 40 次迭代，进行一次可视化渲染
    if iteration % 40 == 0:
        # Visualize only a single randomly selected element of the batch.
        im_show_idx = int(torch.randint(low=0, high=batch_size, size=(1,)))
        fig, ax = plt.subplots(2, 2, figsize=(10, 10))
        ax = ax.ravel()
        clamp_and_detach = lambda x: x.clamp(0.0, 1.0).cpu().detach().numpy()
        ax[0].imshow(clamp_and_detach(rendered_images[im_show_idx]))
        ax[1].imshow(clamp_and_detach(target_images[batch_idx[im_show_idx], ..., :3]))
        ax[2].imshow(clamp_and_detach(rendered_silhouettes[im_show_idx, ..., 0]))
        ax[3].imshow(clamp_and_detach(target_silhouettes[batch_idx[im_show_idx]]))
        for ax_, title_ in zip(
            ax, 
            ("rendered image", "target image", "rendered silhouette", "target silhouette")
        ):
            ax_.grid("off")
            ax_.axis("off")
            ax_.set_title(title_)
        fig.canvas.draw(); fig.show()
        display.clear_output(wait=True)
        display.display(fig)

## 5. 将优化后的场景体积进行可视化
最后，旋转场景体积的 y 轴，从多个视点进行渲染，将优化后的体积进行可视化。

In [ ]:
def generate_rotating_volume(volume_model, n_frames = 50):
    logRs = torch.zeros(n_frames, 3, device=device)
    logRs[:, 1] = torch.linspace(0.0, 2.0 * 3.14, n_frames, device=device)
    Rs = so3_exponential_map(logRs)
    Ts = torch.zeros(n_frames, 3, device=device)
    Ts[:, 2] = 2.7
    frames = []
    print('Generating rotating volume ...')
    for R, T in zip(tqdm(Rs), Ts):
        camera = FoVPerspectiveCameras(
            R=R[None], 
            T=T[None], 
            znear = target_cameras.znear[0],
            zfar = target_cameras.zfar[0],
            aspect_ratio = target_cameras.aspect_ratio[0],
            fov = target_cameras.fov[0],
            device=device,
        )
        frames.append(volume_model(camera)[..., :3].clamp(0.0, 1.0))
    return torch.cat(frames)
    
with torch.no_grad():
    rotating_volume_frames = generate_rotating_volume(volume_model, n_frames=7*4)

image_grid(rotating_volume_frames.clamp(0., 1.).cpu().numpy(), rows=4, cols=7, rgb=True, fill=True)
plt.show()

## 6. 结论

本教程中演示了如何优化场景的 3D 立体构造，使已知视点的体积渲染与每个视点的观测图像相匹配。教程中的渲染是使用 NDCGridRaysampler 和 EmissionAbsorptionRaymarcher 构成的 PyTorch3D 立体渲染器完成的。